In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from timm.models.layers import DropPath
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import matplotlib.pyplot as plt

# 1) Reproducibility & Device Setup

def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 100

# 2) Load & Filter Data
train_csv = '/kaggle/input/smai-project/Phase_2_data/labels_train.csv'
val_csv   = '/kaggle/input/smai-project/Phase_2_data/labels_val.csv'
train_dir = '/kaggle/input/smai-project/Phase_2_data/images_train'
val_dir   = '/kaggle/input/smai-project/Phase_2_data/images_val'
exclude_val_ids = [95, 145, 146, 158, 159, 160, 161]

train_df = pd.read_csv(train_csv)
# filter coordinates outside desired range
train_df = train_df[train_df.latitude.between(200000,230000) & 
                    train_df.longitude.between(140000,150000)].reset_index(drop=True)
# compute standardization params
mu = train_df[['latitude','longitude']].mean().values
sigma = train_df[['latitude','longitude']].std().values
# create tensors for denormalization on device
mu_tensor = torch.tensor(mu, dtype=torch.float32, device=device)
sigma_tensor = torch.tensor(sigma, dtype=torch.float32, device=device)

# # 3) Optional EDA: scatter of normalized coords by region
# train_df['lat_n'] = (train_df.latitude - mu[0]) / sigma[0]
# train_df['lon_n'] = (train_df.longitude - mu[1]) / sigma[1]
# plt.figure(figsize=(8,6))
# plt.scatter(train_df.lon_n, train_df.lat_n, c=train_df.region_id, s=5, cmap='tab20')
# plt.xlabel('Norm Lon'); plt.ylabel('Norm Lat'); plt.title('Normalized Coordinates by Region')
# plt.colorbar(label='region_id'); plt.savefig('latlon_scatter.png',dpi=300); plt.close()

# 4) Dataset & Normalization
class GeoCoordDataset(Dataset):
    def __init__(self, df, img_dir, mu, sigma, transforms=None):
        self.df = df
        self.img_dir = img_dir
        self.mu = torch.tensor(mu, dtype=torch.float32)
        self.sigma = torch.tensor(sigma, dtype=torch.float32)
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(os.path.join(self.img_dir, row.filename)).convert('RGB')
        if self.transforms:
            img = self.transforms(img)
        raw = torch.tensor([row.latitude, row.longitude], dtype=torch.float32)
        normed = (raw - self.mu) / self.sigma
        return img, normed, raw, row.filename

# 5) Transforms & DataLoaders
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224, scale=(0.7,1.0)),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ColorJitter(0.3,0.3,0.3,0.15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

dataset_train = GeoCoordDataset(train_df, train_dir, mu, sigma, transforms=train_transform)
loader_train = DataLoader(dataset_train, batch_size=4, shuffle=True, num_workers=4)
val_df = pd.read_csv(val_csv)
val_df = val_df[~val_df.index.isin(exclude_val_ids)].reset_index(drop=True)
dataset_val = GeoCoordDataset(val_df, val_dir, mu, sigma, transforms=val_transform)
loader_val   = DataLoader(dataset_val, batch_size=4, shuffle=False, num_workers=4)

# 6) Model Architecture (CoordModel)
def freeze_module(module):
    for p in module.parameters(): p.requires_grad = False

class CoordModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = torch.hub.load('facebookresearch/dinov2','dinov2_vitb14',pretrained=True)
        embed_dim = self.backbone.embed_dim
        freeze_module(self.backbone.patch_embed)
        for blk in self.backbone.blocks[:6]: freeze_module(blk)
        for blk in self.backbone.blocks[6:]: blk.drop_path = DropPath(0.1)
        self.head = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(embed_dim, embed_dim//2),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(embed_dim//2, 2)
        )

    def forward(self, x):
        features = self.backbone(x)
        return self.head(features)

model = CoordModel().to(device)

# 7) Losses & Metrics
criterion = nn.SmoothL1Loss()
mse_loss = nn.MSELoss()

def compute_denorm_mse(pred_norm, raw_gt, mu_t, sigma_t):
    pred = pred_norm * sigma_t + mu_t
    mse_per_dim = ((pred - raw_gt)**2).mean(dim=0)
    return mse_per_dim[0].item(), mse_per_dim[1].item(), mse_per_dim.mean().item()

# 8) Optimizer & Scheduler
optimizer = optim.AdamW([
    {'params':[p for p in model.backbone.parameters() if p.requires_grad], 'lr':2e-5},
    {'params':model.head.parameters(), 'lr':2e-3}
], weight_decay=1e-2)
scheduler = CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-7)
scaler = GradScaler()

# 9) Training & Validation Loops
def train_one_epoch(epoch):
    model.train(); running_loss=0.0; sample_count=0
    pbar = tqdm(loader_train, desc=f"Train {epoch+1}")
    for imgs, t_norm, _, _ in pbar:
        imgs, t_norm = imgs.to(device), t_norm.to(device)
        with autocast(): pred_norm = model(imgs); loss = criterion(pred_norm, t_norm)
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer); nn.utils.clip_grad_norm_(model.parameters(),1.0)
        scaler.step(optimizer); scaler.update(); optimizer.zero_grad()
        bs = imgs.size(0); running_loss += loss.item()*bs; sample_count += bs
        pbar.set_postfix({'norm_loss':f"{running_loss/sample_count:.4f}"})
    scheduler.step()


def validate(epoch):
    model.eval(); total_norm=0.0; total_lat=0.0; total_lon=0.0; count=0
    preds=[]; fnames=[]
    pbar = tqdm(loader_val, desc=f"Val {epoch+1}")
    for imgs, t_norm, raw, names in pbar:
        imgs, t_norm, raw = imgs.to(device), t_norm.to(device), raw.to(device)
        with torch.no_grad(): pred_norm = model(imgs)
        bs = imgs.size(0)
        total_norm += criterion(pred_norm, t_norm).item()*bs
        lat_m, lon_m, _ = compute_denorm_mse(pred_norm, raw, mu_tensor, sigma_tensor)
        total_lat += lat_m*bs; total_lon += lon_m*bs; count += bs
        preds.append(pred_norm)  # keep on GPU
        fnames.extend(names)
        pbar.set_postfix({'denorm_avg_mse':f"{(total_lat+total_lon)/(2*count):.4f}"})
    total_norm /= count; total_lat /= count; total_lon /= count
    avg_mse = (total_lat+total_lon)/2
    print(f"Epoch {epoch+1}: Val norm_loss {total_norm:.4f}, Lat MSE {total_lat:.4f}, Lon MSE {total_lon:.4f}, Avg MSE {avg_mse:.4f}")
    # denormalize all predictions on GPU, then move to CPU
    all_pred_norm = torch.cat(preds)
    denorm = all_pred_norm * sigma_tensor.unsqueeze(0) + mu_tensor.unsqueeze(0)
    denorm_cpu = denorm.cpu().numpy()
    df = pd.DataFrame({'filename':fnames, 'pred_latitude':denorm_cpu[:,0], 'pred_longitude':denorm_cpu[:,1]})
    df.to_csv('best_val_preds.csv', index=False)
    return total_norm, total_lat, total_lon, avg_mse

# 10) Main Training Loop
best_mse = float('inf')
for epoch in range(epochs):
    train_one_epoch(epoch)
    _, lat_mse, lon_mse, avg_mse = validate(epoch)
    if avg_mse < best_mse:
        best_mse = avg_mse
        torch.save(model.state_dict(), 'best_coord_model.pth')
print(f"Best Val Avg MSE: {best_mse:.4f}")


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb

Epoch 1: Val norm_loss 0.1588, Lat MSE 265572.6290, Lon MSE 512910.6604, Avg MSE 389241.6447


Val 2: 100%|██████████| 91/91 [00:07<00:00, 11.67it/s, denorm_avg_mse=213867.3997]


Epoch 2: Val norm_loss 0.0896, Lat MSE 168836.0067, Lon MSE 258898.7928, Avg MSE 213867.3997


Val 3: 100%|██████████| 91/91 [00:07<00:00, 11.79it/s, denorm_avg_mse=155240.7199]


...

Epoch 98: Val norm_loss 0.0119, Lat MSE 23779.8027, Lon MSE 28093.1389, Avg MSE 25936.4708


Val 99: 100%|██████████| 91/91 [00:07<00:00, 11.76it/s, denorm_avg_mse=33092.3496]


Epoch 99: Val norm_loss 0.0139, Lat MSE 34486.0515, Lon MSE 31698.6477, Avg MSE 33092.3496


Val 100: 100%|██████████| 91/91 [00:07<00:00, 11.81it/s, denorm_avg_mse=36641.3330]

Epoch 100: Val norm_loss 0.0147, Lat MSE 38836.2225, Lon MSE 34446.4435, Avg MSE 36641.3330
Best Val Avg MSE: 18666.6941
